In [9]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import pandas as pd
import plotly.express as px
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime
import plotly.graph_objects as go



In [10]:
import pandas as pd
import numpy as np
import holidays

# Load the data
data = pd.read_csv("performance.csv")
# data = data[data["Reg"] != "Pospay"]

# Generate a list of Indonesian holidays for 2023-2025
indonesia_holidays = holidays.Indonesia(years=[2023, 2024, 2025])
holidays_set = set(indonesia_holidays.keys())

# Function to check if a date is a holiday
def check_date(row, holidays_set):
    return row['Day of Date'].date() in holidays_set  

# List to hold dataframes for each product segment
segment_dfs = []

# Loop over each unique product segment
for segment in data['Segmen Produk'].unique():
    segment_data = data[data["Segmen Produk"] == segment]
    segment_data['Day of Date'] = pd.to_datetime(segment_data['Day of Date'])
    segment_data['Revenue'] = segment_data['Revenue'].clip(lower=0)
    segment_data['Target'] = segment_data['Target'].clip(lower=0)

    # Create a complete date range
    all_dates = pd.date_range(start=segment_data['Day of Date'].min(), end=segment_data['Day of Date'].max())
    segment_data = segment_data.groupby('Day of Date')["Target"].sum().reset_index()
    segment_data = segment_data.set_index('Day of Date').reindex(all_dates).fillna(0).reset_index()
    segment_data.rename(columns={'index': 'Day of Date'}, inplace=True)

    # Extract date-based features
    segment_data['Week of Year'] = segment_data['Day of Date'].dt.isocalendar().week
    segment_data['Week of Month'] = (segment_data['Day of Date'].dt.day - 1) // 7 + 1
    segment_data['Day of Year'] = segment_data['Day of Date'].dt.dayofyear
    segment_data['Day of Month'] = segment_data['Day of Date'].dt.day
    segment_data['Month of Year'] = segment_data['Day of Date'].dt.month
    segment_data['Year'] = segment_data['Day of Date'].dt.year
    segment_data['holiday'] = segment_data.apply(lambda row: check_date(row, holidays_set), axis=1)

    # Cyclic Encoding for Seasonality
    segment_data['month_sin'] = np.sin(2 * np.pi * segment_data['Month of Year'] / 12)
    segment_data['month_cos'] = np.cos(2 * np.pi * segment_data['Month of Year'] / 12)
    segment_data['dayofyear_sin'] = np.sin(2 * np.pi * segment_data['Day of Year'] / 365)
    segment_data['dayofyear_cos'] = np.cos(2 * np.pi * segment_data['Day of Year'] / 365)

    # Lag Features on Target
    segment_data['lag_target_7'] = segment_data['Target'].shift(7)
    segment_data['lag_target_30'] = segment_data['Target'].shift(30)
    
    # Rolling Averages on Target
    segment_data['rolling_target_7'] = segment_data['Target'].rolling(7).mean()
    segment_data['rolling_target_30'] = segment_data['Target'].rolling(30).mean()
    
    # Difference Feature on Target
    segment_data['diff_target'] = segment_data['Target'].diff()
    
    # Holiday Proximity Features
    segment_data['days_since_last_holiday'] = (segment_data['Day of Date'] - segment_data['Day of Date'][segment_data['holiday']].shift(1)).dt.days.fillna(0)
    segment_data['days_until_next_holiday'] = (segment_data['Day of Date'][segment_data['holiday']].shift(-1) - segment_data['Day of Date']).dt.days.fillna(0)

    segment_data['weekend'] = segment_data['Day of Date'].dt.weekday >= 5
    segment_data['days_to_holiday'] = segment_data.apply(
        lambda row: min(abs((row['Day of Date'] - pd.to_datetime(h)).days) for h in holidays_set), axis=1
    )
    
    # Fill NaN values for new features
    segment_data.fillna(0, inplace=True)
    
    # Sort values by date
    segment_data = segment_data.sort_values(by='Day of Date').reset_index(drop=True)

    # Define split dates
    split_date_test = pd.to_datetime('2024-11-30')
    split_date_test_up = pd.to_datetime('2024-12-31')
    split_date_up = pd.to_datetime('2024-12-31')

    # Split the data
    train = segment_data[segment_data["Day of Date"] <= split_date_test]
    test = segment_data[(segment_data["Day of Date"] >= split_date_test) & (segment_data["Day of Date"] <= split_date_test_up)]
    forecast = segment_data[(segment_data["Day of Date"] >= pd.to_datetime('2024-01-01')) & (segment_data["Day of Date"] <= split_date_up)]

    # Select columns for model training and testing
    train_date = train["Day of Date"]
    train = train.drop(columns=["Day of Date"])
    test_date = test["Day of Date"]
    test = test.drop(columns=["Day of Date"])
    forecast_date = forecast["Day of Date"]
    forecast = forecast.drop(columns=["Day of Date"])

    # Store the processed DataFrame in the list
    segment_dfs.append((segment, train, test, forecast))

# Now segment_dfs contains processed data for each segment with no missing dates and added features


In [12]:
# Function to remove highly correlated features
def remove_highly_correlated_features(df, target_column='Revenue', threshold=0.6):
    # Compute the correlation matrix
    corr_matrix = df.corr().abs()
    
    # Select the upper triangle of the correlation matrix
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than the threshold, except 'Revenue'
    to_drop = []
    for column in upper_triangle.columns:
        if column == target_column:
            continue
        
        # Find features correlated above the threshold
        high_corr_features = upper_triangle[column][upper_triangle[column] > threshold].index.tolist()
        
        # If 'Revenue' is among the highly correlated, retain both
        if target_column in high_corr_features:
            continue  # Skip dropping this column as it is correlated with 'Revenue' and we want to keep both
        
        # Drop columns that have high correlation with any other feature (except 'Revenue')
        if any(upper_triangle[column] > threshold):
            to_drop.append(column)
    
    # Drop the columns from the dataframe, but keep 'Revenue'
    return df.drop(columns=to_drop)

# Function to apply feature removal for each segment in segment_dfs
def remove_correlated_features_for_segments(segment_dfs, target_column='Revenue', threshold=0.6):
    # List to hold processed data for each segment
    processed_segments = []

    # Loop through each segment
    for segment, train, test, forecast in segment_dfs:
        # Apply the feature removal function to the train dataset
        train_filtered = remove_highly_correlated_features(train, target_column=target_column, threshold=threshold)
        
        # Ensure test and forecast datasets have the same columns as filtered train
        test_filtered = test[train_filtered.columns]
        forecast_filtered = forecast[train_filtered.columns]
        
        # Store the processed data for this segment
        processed_segments.append((segment, train_filtered, test_filtered, forecast_filtered))
        
        # Print the columns that were kept for this segment
        print(f"Columns kept in train after removing correlated features for segment '{segment}':", train_filtered.columns.tolist())

    return processed_segments

# Example usage with segment_dfs
# Assuming segment_dfs contains the segmented data for each product, with the structure:
# [(segment_name1, train1, test1, forecast1), (segment_name2, train2, test2, forecast2), ...]

# Apply the function to segment_dfs
processed_segment_dfs = remove_correlated_features_for_segments(segment_dfs, target_column='Revenue', threshold=0.8)

Columns kept in train after removing correlated features for segment 'Bendapos & Meterai': ['Target', 'Week of Year', 'Week of Month', 'Year', 'holiday', 'month_cos', 'lag_target_30', 'diff_target', 'days_since_last_holiday', 'days_until_next_holiday', 'weekend', 'days_to_holiday']
Columns kept in train after removing correlated features for segment 'Fronting': ['Target', 'Week of Year', 'Week of Month', 'Year', 'holiday', 'month_cos', 'lag_target_7', 'lag_target_30', 'diff_target', 'days_since_last_holiday', 'days_until_next_holiday', 'weekend', 'days_to_holiday']
Columns kept in train after removing correlated features for segment 'Fund Distribution': ['Target', 'Week of Year', 'Week of Month', 'Year', 'holiday', 'month_cos', 'lag_target_7', 'lag_target_30', 'rolling_target_7', 'diff_target', 'days_since_last_holiday', 'days_until_next_holiday', 'weekend', 'days_to_holiday']
Columns kept in train after removing correlated features for segment 'Payment': ['Target', 'Week of Year', 'We

In [135]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Function to train and evaluate the model for each product segment
def train_and_evaluate_model(segment_dfs):
    models = {}
    evaluation_summary = []  

    for segment, train, test,forecast in segment_dfs:
        print(f'Calculating {segment}')

        X_train = train.drop(columns=['Revenue'])
        y_train = train['Revenue']
        X_test = test.drop(columns=['Revenue'])
        y_test = test['Revenue']

        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            learning_rate=0.05,  # Increased learning rate
            n_estimators=1200,  # Lowering tree count
            max_depth=3,  # Reducing tree complexity
            min_child_weight=12,
            subsample=0.7,
            colsample_bytree=0.8,
            gamma=10,
            reg_alpha=10,  # Stronger L1 regularization
            reg_lambda=15,  # Stronger L2 regularization
            eval_metric='rmse',
            tree_method='hist',
            early_stopping_rounds=100,  # Faster stopping
            verbosity=0
        )


        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Train & Test Errors
        train_mae = mean_absolute_error(y_train, y_train_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
        test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)

        evaluation_summary.append({
            'Segment': segment,
            'Train_MAE': train_mae,
            'Test_MAE': test_mae,
            'Train_RMSE': train_rmse,
            'Test_RMSE': test_rmse,
            'Train_R2': train_r2,
            'Test_R2': test_r2
        })

        models[segment] = model

    return models, evaluation_summary


# Example usage
models, eval_summary = train_and_evaluate_model(processed_segment_dfs)

# # Print evaluation summary
# for summary in eval_summary:
#     print(f"Segment '{summary['Segment']}': Best_n_estimators = {summary['Best_n_estimators']}, "
#           f"MAE = {summary['Mean_Absolute_Error']:.4f}, MSE = {summary['Mean_Squared_Error']:.4f}, "
#           f"R^2 Score = {summary['R2_Score']:.4f}")


Calculating Bendapos & Meterai


KeyError: "['Revenue'] not found in axis"

In [118]:
eval_summary

[{'Segment': 'Bendapos & Meterai',
  'Train_MAE': 14278669.052985491,
  'Test_MAE': 86310322.703125,
  'Train_RMSE': 20645541.79112616,
  'Test_RMSE': 161178164.78528243,
  'Train_R2': 0.9976931574043246,
  'Test_R2': 0.8966460490716848},
 {'Segment': 'Fronting',
  'Train_MAE': 47409684.137410715,
  'Test_MAE': 125585895.55664062,
  'Train_RMSE': 149911201.50008684,
  'Test_RMSE': 226699695.8102973,
  'Train_R2': 0.9931007514783076,
  'Test_R2': 0.9914360591995779},
 {'Segment': 'Fund Distribution',
  'Train_MAE': 142625984.37526786,
  'Test_MAE': 720173919.390625,
  'Train_RMSE': 506392013.5808411,
  'Test_RMSE': 2776321276.7876577,
  'Train_R2': 0.9892290049973674,
  'Test_R2': 0.9371815708544516},
 {'Segment': 'Payment',
  'Train_MAE': 28351602.493571427,
  'Test_MAE': 51826272.5625,
  'Train_RMSE': 65187979.26317384,
  'Test_RMSE': 94511105.42483084,
  'Train_R2': 0.9780869814733697,
  'Test_R2': 0.9615954576202782},
 {'Segment': 'Remittance & Sharia',
  'Train_MAE': 6637752.050273

In [119]:
import plotly.express as px

# Function to plot predictions for all segments
def plot_predictions(models, segment_dfs):
    for segment, train, test,forecast in segment_dfs:
        # Retrieve the model for the current segment
        model = models[segment]
        
        # Prepare the test data
        X_test = test.drop(columns=['Revenue'])
        y_test = test['Revenue']  # Assuming this column exists

        # Make predictions on the test set
        predictions_test = model.predict(X_test)
        
        # Create the scatter plot
        fig = px.scatter(x=test_date, y=predictions_test,
                         labels={
                             'x': "Tanggal",
                             'y': "Prediksi Penjualan"},
                         title=f"<b> Prediksi Performa untuk {segment} </b>")
        
        # Update the layout with actual sales data
        fig.update_layout(title_font=dict(size=16), title_x=0.5)
        fig.add_scatter(x=test_date, y=y_test, name='Actual Penjualan')
        fig.add_scatter(x=test_date, y=predictions_test, name='Prediksi Penjualan')
        fig.update_layout(showlegend=True)
        
        # Show the plot
        fig.show()

# Example usage
plot_predictions(models, processed_segment_dfs)


In [120]:
import plotly.express as px
import pandas as pd

# Function to plot predictions for all segments
def plot_predictions(models, segment_dfs):
    for segment, train, test, forecast in segment_dfs:
        # Retrieve the model for the current segment
        model = models[segment]

        # Extract test and forecast dates
        test['Date'] = test_date
        train['Date'] = train_date
        forecast['Date'] = forecast_date

        # Split test data into two parts: Jan-Nov & Dec
        test_jan_nov = train[(train['Date'] < '2024-12-01') & (train['Date'] >= '2024-01-01')]
        test_dec = test[test["Date"]>= '2024-12-01']  # (You might want to check if using `train` here is intentional)

        # Prepare data for prediction (remove 'Predicted' column if it exists)
        feature_columns = train.columns.drop(['Date', 'Revenue'], errors='ignore')  # Keep only the filtered features
        
        X_test_jan_nov = test_jan_nov[feature_columns]
        X_test_dec = test_dec[feature_columns]
        X_forecast = forecast[feature_columns]

        # Make predictions
        test_jan_nov['Predicted'] = model.predict(X_test_jan_nov)
        test_dec['Predicted'] = model.predict(X_test_dec)
        forecast['Predicted'] = model.predict(X_forecast)
        
        # Plot Jan-Nov test predictions
        fig = px.scatter(x=test_jan_nov['Date'], y=test_jan_nov['Predicted'],
                         labels={'x': "Tanggal", 'y': "Prediksi Penjualan"},
                         title=f"<b> Prediksi Performa untuk {segment} </b>")
        fig.add_scatter(x=test_jan_nov['Date'], y=test_jan_nov['Revenue'], name='Actual Penjualan (jan - Nov)')

        # Add December test data
        fig.add_scatter(x=test_dec['Date'], y=test_dec['Revenue'], name='Actual Penjualan (Des)')

        # Add full-year forecast
        fig.add_scatter(x=forecast['Date'], y=forecast['Predicted'], name='Prediksi 2024 (Forecast)', mode='lines')

        # Show the plot
        fig.update_layout(title_font=dict(size=16), title_x=0.5, showlegend=True)
        fig.show()

        test_jan_nov['Month of Year'] = test_jan_nov['Date'].dt.month
        test_dec['Month of Year'] = test_dec['Date'].dt.month
        forecast['Month of Year'] = forecast['Date'].dt.month

        #monthly bar
        monthly_train = test_jan_nov.groupby('Month of Year')['Revenue'].sum().reset_index()
        monthly_test = test_dec.groupby('Month of Year')['Revenue'].sum().reset_index()
        forecast_monthly = forecast.groupby('Month of Year')['Predicted'].sum().reset_index()
        #bar
        fig2 = px.bar(monthly_train, x='Month of Year', y='Revenue', 
                    labels={'x': "Bulan", 'y': "Penjualan"}, 
                    title=f"<b> Penjualan Bulanan untuk {segment} </b>")

        fig2.add_bar(x=monthly_test['Month of Year'], y=monthly_test['Revenue'], name='Actual Penjualan (Des)')
        fig2.add_bar(x=forecast_monthly['Month of Year'], y=forecast_monthly['Predicted'], name='Prediksi 2024 (Forecast)')

        fig2.update_layout(barmode='group')  # Ensures bars are side by side for comparison
        fig2.show()

# Example usage
plot_predictions(models, processed_segment_dfs)


KeyError: 'Day of Date'

In [126]:
forecast

,Target,Revenue,Week of Year,Week of Month,Day of Year,Day of Month,Month of Year,Year,holiday,month_sin,...,rolling_target_30,ema_7,ema_30,diff_revenue,diff_target,days_since_last_holiday,days_until_next_holiday,weekend,revenue_anomaly,days_to_holiday
365,480509397.0,0.000000e+00,1,1,1,1,1,2024,True,5.000000e-01,...,1.601698e+07,4.320450e+08,7.288027e+08,-1081898.0,480509397.0,7.0,38.0,False,False,0
366,691736449.0,6.623427e+08,1,1,2,2,1,2024,False,5.000000e-01,...,3.907486e+07,4.896194e+08,7.245149e+08,662342688.0,211227052.0,0.0,0.0,False,False,1
367,826778325.0,9.787994e+08,1,1,3,3,1,2024,False,5.000000e-01,...,6.663414e+07,6.119144e+08,7.409204e+08,316456662.0,135041876.0,0.0,0.0,False,False,2
368,821658442.0,1.155530e+09,1,1,4,4,1,2024,False,5.000000e-01,...,9.402275e+07,7.478182e+08,7.676693e+08,176730308.0,-5119883.0,0.0,0.0,False,False,3
369,767503305.0,1.012583e+09,1,1,5,5,1,2024,False,5.000000e-01,...,1.196062e+08,8.140095e+08,7.834702e+08,-142946476.0,-54155137.0,0.0,0.0,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,681486016.0,7.879836e+08,52,4,362,27,12,2024,False,-2.449294e-16,...,6.618394e+08,6.851510e+08,8.019643e+08,150579938.0,-52235241.0,0.0,0.0,False,False,2
727,470007565.0,6.022855e+08,52,4,363,28,12,2024,False,-2.449294e-16,...,6.350204e+08,6.644346e+08,7.890818e+08,-185698144.0,-211478451.0,0.0,0.0,True,False,3
728,714006.0,0.000000e+00,52,5,364,29,12,2024,False,-2.449294e-16,...,5.947694e+08,4.983260e+08,7.381733e+08,-602285486.0,-469293559.0,0.0,0.0,True,False,3
729,436036534.0,9.159059e+08,1,5,365,30,12,2024,False,-2.449294e-16,...,5.714292e+08,6.027210e+08,7.496399e+08,915905916.0,435322528.0,0.0,0.0,False,False,2
